# Fashion MNIST

### 데이터 가져오기

In [1]:
import tensorflow as tf

data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

4422102/4422102 [==============================] - 0s 0us/step


### Normalization (정규화)

이미지 픽셀 값을 0 ~ 1 사이의 값으로 바꿈

In [2]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

### 신경망 정의

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

### Loss function & Optimizer

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

sparse_categorical_crossentropy : 하나의 숫자를 예측하는 것이 아니라 하나의 카테고리를 선택  

`metrics=['accuracy']` : loss 와 accuracy 동시 출력

### Train

In [6]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 2ms/step - loss: 0.4987 - accuracy: 0.8263
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3749 - accuracy: 0.8641
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3370 - accuracy: 0.8765
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3127 - accuracy: 0.8834
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2960 - accuracy: 0.8906


### Evaluate

In [7]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.3698 - accuracy: 0.8692


[0.3698198199272156, 0.8691999912261963]

모델 정확도 = 86.92%

### Predict

In [8]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

313/313 [==============================] - 1s 2ms/step
[1.2457368e-06 1.9189658e-08 6.2559218e-07 4.5600935e-08 3.8794947e-06
 1.2112443e-03 8.9519399e-06 2.8383078e-02 2.2114384e-05 9.7036874e-01]
9


첫 번째 의류 아이템의 실제 label 은 9 이기 때문에 배열의 마지막 값이 가장 작은 것을 확인할 수 있다.  

즉, 97.0% 확률로 label 9 이라고 출력한 것이다.

### Overfitting (과대 적합)

모델이 훈련 세트에 과하게 적합한 상태가 되어 **일반성이 떨어지는** 현상

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=50)

Epoch 1/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4998 - accuracy: 0.8240
Epoch 2/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3743 - accuracy: 0.8649
Epoch 3/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3386 - accuracy: 0.8769
Epoch 4/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3134 - accuracy: 0.8849
Epoch 5/50
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2967 - accuracy: 0.8906
Epoch 6/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2811 - accuracy: 0.8960
Epoch 7/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2680 - accuracy: 0.9010
Epoch 8/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2594 - accuracy: 0.9036
Epoch 9/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2454 - accuracy: 0.9083
Epoch 10/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.238

In [11]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.4984 - accuracy: 0.8871


[0.4983988404273987, 0.8870999813079834]

Train-set 의 정확도는 크게 높아졌지만 Test-set 에 대한 정확도는 조금만 향상되었다.  

오래 훈련했기 때문에 더 높은 성능이 나오긴 했지만 항상 이러한 결과가 나오지는 않는다.

### Train 조기 종료

원하는 정확도에 도달하면 훈련을 멈추게 하는 방법 → Callback 콜백 사용

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):       # epoch가 끝날 때마다
        if(logs.get('accuracy') > 0.90):            # 정확도를 체크
            print("\n정확도 90%에 도달하여 훈련을 멈춥니다!")
            self.model.stop_training = True

In [17]:
callbacks = myCallback()

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=50, callbacks=[callbacks])

Epoch 1/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4964 - accuracy: 0.8253
Epoch 2/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3732 - accuracy: 0.8659
Epoch 3/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3343 - accuracy: 0.8785
Epoch 4/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3113 - accuracy: 0.8864
Epoch 5/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2955 - accuracy: 0.8917
Epoch 6/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2788 - accuracy: 0.8975
Epoch 7/50
1853/1875 [============================>.] - ETA: 0s - loss: 0.2666 - accuracy: 0.9008
정확도 90%에 도달하여 훈련을 멈춥니다!
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2669 - accuracy: 0.9007
